In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import json
import cv2

In [ ]:
from pycocotools.coco import COCO

In [ ]:
TRAIN_PATH = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/train'
VAL_PATH = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/valid'
TEST_PATH = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/test'

TRAIN_ANN = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/train/_annotations.coco.json'
VAL_ANN = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/valid/_annotations.coco.json'
TEST_ANN = '/kaggle/input/brain-tumor-image-dataset-semantic-segmentation/test/_annotations.coco.json'

In [ ]:
print(os.listdir(TRAIN_PATH)[:5])

In [ ]:
image_sample = Image.open(TRAIN_PATH + "/" + os.listdir(TRAIN_PATH)[0])
plt.imshow(image_sample)

In [ ]:
with open(TRAIN_ANN, "r") as f:
    raw_coco = f.read()
    coco = json.loads(raw_coco)
    print(coco.keys())

In [ ]:
print(coco["images"][100])

In [ ]:
print(coco["annotations"][100])

In [ ]:
def image_id_to_image_and_mask(image_id):
    image_path = coco["images"][image_id]["file_name"]
    image = np.array(Image.open(TRAIN_PATH + "/" + image_path))

    bboxes = []
    for ann in coco["annotations"]:
        if ann["image_id"] == image_id:
            bboxes.append(ann["bbox"])
    mask = np.zeros((coco["images"][image_id]["height"], coco["images"][image_id]["width"]))
    for bbox in bboxes:
        x_min, y_min, width, height = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        mask[y_min:y_min + height, x_min:x_min + width] = 255 

    return image, mask

In [ ]:
image, mask = image_id_to_image_and_mask(0)
plt.subplot(1, 2, 1)
plt.imshow(Image.fromarray(image))
plt.title("image")

plt.subplot(1, 2, 2)
plt.imshow(Image.fromarray(mask))
plt.title("mask")

In [ ]:
import pathlib

import torch
import torch.utils.data

from torchvision import models, datasets, tv_tensors
from torchvision.transforms import v2

In [ ]:
transforms = v2.Compose(
    [
        v2.ToImage(),
        v2.RandomPhotometricDistort(p=1),
        v2.RandomZoomOut(fill={tv_tensors.Image: (123, 117, 104), "others": 0}),
        v2.RandomIoUCrop(),
        v2.RandomHorizontalFlip(p=1),
        v2.SanitizeBoundingBoxes(),
        v2.ToDtype(torch.float32, scale=True),
    ]
)

dataset = datasets.CocoDetection(TRAIN_PATH, TRAIN_ANN, transforms=transforms)
dataset = datasets.wrap_dataset_for_transforms_v2(dataset, target_keys=["boxes", "labels", "masks"])

In [ ]:
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=4,
    # We need a custom collation function here, since the object detection
    # models expect a sequence of images and target dictionaries. The default
    # collation function tries to torch.stack() the individual elements,
    # which fails in general for object detection, because the number of bounding
    # boxes varies between the images of the same batch.
    collate_fn=lambda batch: tuple(zip(*batch)),
)

model = models.get_model("maskrcnn_resnet50_fpn_v2", weights=None, weights_backbone=None).train()

for imgs, targets in data_loader:
    loss_dict = model(imgs, targets)
    # Put your training logic here

    print(f"{[img.shape for img in imgs] = }")
    print(f"{[type(target) for target in targets] = }")
    for name, loss_val in loss_dict.items():
        print(f"{name:<20}{loss_val:.3f}")

In [ ]:
torch.save(model, "/kaggle/working/weights.pth")